In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [56]:
#importing file
donors_num = pd.read_csv(r"C:\Users\pedro\Desktop\Ironhack\numerical.csv")
donors_target = pd.read_csv(r"C:\Users\pedro\Desktop\Ironhack\target.csv")

In [57]:
#checking null values
def null_percentage(df):
    total_null = df.isnull().sum()
    total_rows = len(df)
    percentage_null = (total_null / total_rows) * 100
    null_df = pd.DataFrame({
        'Total Null Values': total_null,
        'Percentage Null': percentage_null
    })
    
    null_df = null_df.sort_values(by='Percentage Null', ascending=False)
    return null_df


null_percentages = null_percentage(donors_num)
print(null_percentages)

          Total Null Values  Percentage Null
TCODE                     0              0.0
EIC4                      0              0.0
EIC11                     0              0.0
EIC10                     0              0.0
EIC9                      0              0.0
...                     ...              ...
ETHC1                     0              0.0
HHD12                     0              0.0
HHD11                     0              0.0
HHD10                     0              0.0
CLUSTER2                  0              0.0

[315 rows x 2 columns]


In [58]:
#concatenate numerical and categorical, dropping TARGET_D 
donors = pd.concat([donors_num, donors_target], axis = 1)
donors = donors.drop(['TARGET_D'], axis = 1)
donors

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2,TARGET_B
0,0,60.000000,5,9,0,0,39,34,18,10,...,5.0,12.0,10.0,4,7.741935,95515,0,4,39,0
1,1,46.000000,6,9,16,0,15,55,11,6,...,10.0,25.0,25.0,18,15.666667,148535,0,2,1,0
2,1,61.611649,3,1,2,0,20,29,33,6,...,2.0,16.0,5.0,12,7.481481,15078,1,4,60,0
3,0,70.000000,1,4,2,0,23,14,31,3,...,2.0,11.0,10.0,9,6.812500,172556,1,4,41,0
4,0,78.000000,3,2,60,1,28,9,53,26,...,3.0,15.0,15.0,14,6.864865,7112,1,2,26,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,1,61.611649,5,9,0,14,36,47,11,7,...,25.0,25.0,25.0,9,25.000000,184568,0,1,12,0
95408,1,48.000000,7,9,1,0,31,43,19,4,...,20.0,20.0,20.0,9,20.000000,122706,1,1,2,0
95409,1,60.000000,5,9,0,0,18,46,20,7,...,3.0,10.0,10.0,3,8.285714,189641,1,3,34,0
95410,0,58.000000,7,9,0,0,28,35,20,9,...,5.0,21.0,18.0,4,12.146341,4693,1,4,11,1


In [59]:
#train-test split
from sklearn.model_selection import train_test_split

X = donors.drop(['TARGET_B'], axis = 1)
Y = donors['TARGET_B']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [60]:
#logistic regression, accuracy
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train,Y_train)
result = LR.score(X_test,Y_test) 
print("The accuracy of the model is = ", round(result,2))

The accuracy of the model is =  0.95


In [61]:
#undersampling
df = pd.concat([X_train,Y_train], axis= 1)

category_0 = df[df['TARGET_B']==0]
category_1 = df[df['TARGET_B']==1]
category_0 = category_0.sample(len(category_1))
category_0

df_undersamp = pd.concat([category_0, category_1], axis = 0)
df_undersamp["TARGET_B"].value_counts()

TARGET_B
0    3865
1    3865
Name: count, dtype: int64

In [62]:
#undersampling, accuracy
X_undersamp = df_undersamp.drop("TARGET_B", axis = 1)
Y_undersamp = df_undersamp["TARGET_B"]

LR_undersamp = LogisticRegression()
LR_undersamp.fit(X_undersamp, Y_undersamp)
result = LR_undersamp.score(X_undersamp, Y_undersamp) 
print("The accuracy of the model is = ", round(result,2))

The accuracy of the model is =  0.54


In [63]:
#oversampling
df = pd.concat([X_train,Y_train], axis = 1)

category_0 = df[df['TARGET_B']==0]
category_1 = df[df['TARGET_B']==1]

from sklearn.utils import resample

category_1 = resample(category_1,
             replace=True,
             n_samples=len(category_0),
             random_state=42)

df_oversamp = pd.concat([category_0, category_1], axis = 0)
df_oversamp["TARGET_B"].value_counts()

TARGET_B
0    72464
1    72464
Name: count, dtype: int64

In [64]:
#oversampling, accuracy
X_oversamp = df_oversamp.drop("TARGET_B", axis = 1)
Y_oversamp = df_oversamp["TARGET_B"]

LR_oversamp = LogisticRegression()
LR_oversamp.fit(X_oversamp, Y_oversamp)
result = LR_oversamp.score(X_oversamp, Y_oversamp) 
print("The accuracy of the model is = ", round(result,2))

The accuracy of the model is =  0.54


In [65]:
#SMOTE
df = pd.concat([X_train,Y_train], axis = 1)

from imblearn.over_sampling import SMOTE
smote = SMOTE()
Y = df['TARGET_B']
X = df.drop(['TARGET_B'], axis=1)
X_sm, Y_sm = smote.fit_resample(X, Y)
Y_sm.value_counts()

TARGET_B
0    72464
1    72464
Name: count, dtype: int64

In [66]:
#SMOTE, accuracy
LR_smote = LogisticRegression()
LR_smote.fit(X_sm, Y_sm)
result = LR_smote.score(X_sm, Y_sm) 
print("The accuracy of the model is = ", round(result,2))

The accuracy of the model is =  0.55
